# Import libraries

In [2]:
import json
import requests
import time
from codecarbon import EmissionsTracker
from bert_score import score

c:\Users\lynvp\moretests\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Prepare variables

In [18]:
prompt_1 = """Large language models (LLMs) are artificial intelligence
(AI) models with complex architecture and a large
number of parameters that have been trained on very large
amounts of text (billions of words). These models, arising from
the field of natural language processing, can generate natural,
human-like writing and have been designed to react to user
input, enabling conversations and customized output according
to prompts. The release of the chatbot ChatGPT by OpenAI1
in late 2022 has rapidly spread this technology to a wide range
of users (GPT means generative pretrained transformer and
denotes the type of large language model used in this chatbot).
Other companies are offering their own apps featuring different
kinds of LLMs, and this technology is also being rapidly
integrated into existing apps and online tools. Because LLMs
will likely become extremely common, the potentially transformative nature of these models has already sparked a lively
debate about their use."""
prompt_2 = ""
prompt_3 = ""

prompts = [
    prompt_1,
    prompt_2,
    prompt_3
]

In [19]:
data = {
    "model": "llama3",
    "prompt": f'Summarize the following text: {prompts[0]}',
}

# Perform query to Ollama (Llama-3-8b model)

In [20]:
tracker = EmissionsTracker(output_file="ollama_llama3_emissions.csv")
tracker.start()

response = requests.post("http://localhost:11434/api/generate", json=data, stream=True)

full_output = ""
for line in response.iter_lines():
    if line:
        json_data = json.loads(line.decode("utf-8"))
        if "response" in json_data:
            full_output += json_data["response"]

emissions = tracker.stop()

print(f"Summary: {full_output}\nCO₂eq: {emissions:.6f} kg")

[codecarbon WARNING @ 21:39:51] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 21:39:51] [setup] RAM Tracking...
[codecarbon INFO @ 21:39:51] [setup] CPU Tracking...
[codecarbon WARNING @ 21:39:52] We saw that you have a 12th Gen Intel(R) Core(TM) i5-12450HX but we don't know it. Please contact us.
[codecarbon WARNING @ 21:39:52] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 21:39:52] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i5-12450HX
[codecarbon WARNING @ 21:39:52] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 21:39:52] [setup] GPU Tracking...
[codecarbon INFO @ 21:39:52] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 21:39:52] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tra

Summary: Large Language Models (LLMs) are AI models trained on vast amounts of text, capable of generating human-like writing and engaging in conversations with users. The release of OpenAI's ChatGPT in 2022 has rapidly spread this technology to a wide audience, with other companies offering their own apps featuring LLMs. As this technology becomes increasingly common, it has sparked debate about its potential impact.
CO₂eq: 0.000070 kg


# Unload model from GPU

In [22]:
requests.post("http://localhost:11434/api/unload-model", json={"model": "llama3", "keep-alive": 0})
print("Model unloaded successfully.")

time.sleep(2)

Model unloaded successfully.


## Validate the quality of the summary

In [24]:
# --- BERTScore Evaluation ---
# Reference summary (your ground truth text)
reference = ["Large language models (LLMs) are advanced AI systems trained on massive text datasets to understand and generate human-like language. Popularized by OpenAI’s ChatGPT in 2022, these models enable interactive and personalized communication. As various companies adopt and integrate LLMs into their tools, their widespread use is prompting significant discussion about their transformative impact and implications."]

# Candidate (model output)
candidate = ["Large Language Models (LLMs) are AI models trained on vast amounts of text, capable of generating human-like writing and engaging in conversations with users. The release of OpenAI's ChatGPT in 2022 has rapidly spread this technology to a wide audience, with other companies offering their own apps featuring LLMs. As this technology becomes increasingly common, it has sparked debate about its potential impact."]

# Compute BERTScore (use model "bert-base-uncased" for English)
P, R, F1 = score(candidate, reference, lang="en", model_type="bert-base-uncased")

print(f"\nBERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")


BERTScore - Precision: 0.7505, Recall: 0.7735, F1: 0.7618


In [4]:
# --- BERTScore Evaluation ---
# Reference summary (your ground truth text)
reference = ["Large language models (LLMs) are advanced AI systems trained on massive text datasets to understand and generate human-like language. Popularized by OpenAI’s ChatGPT in 2022, these models enable interactive and personalized communication. As various companies adopt and integrate LLMs into their tools, their widespread use is prompting significant discussion about their transformative impact and implications."]

# Candidate (model output)
candidate = ["Large Language Models (LLMs) are AI models trained on vast amounts of text, capable of generating human-like writing and engaging in conversations with users. The release of OpenAI's ChatGPT in 2022 has rapidly spread this technology to a wide audience, with other companies offering their own apps featuring LLMs. As this technology becomes increasingly common, it has sparked debate about its potential impact."]

# Compute BERTScore (use model "bert-base-uncased" for English)
P, R, F1 = score(candidate, reference, lang="en", model_type="microsoft/deberta-xlarge-mnli", device="cuda")

print(f"\nBERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

KeyboardInterrupt: 